### Resposta que gostaríamos do modelo

```
{
  "resposta": "Equação quadrática é uma equação polinomial de segundo grau, com a forma ax² + bx + c = 0.",
  "categoria": "Matemática",
  "fonte": "GascBot"
}
```

### Resposta que o modelo está nos dando

Uma equação quadrática é uma equação polinomial de segundo grau, ou seja, uma equação na forma ax^2 + bx + c = 0, onde a, b e c são constantes e a é diferente de zero. A incógnita da equação é x e o objetivo é encontrar os valores de x que satisfazem a equação. As equações quadráticas podem ter duas soluções reais, uma solução real ou duas soluções complexas.

### Prompt para que ele nos dê a mensagem formatada como queremos

system_mes = '''
Responda as perguntas em um parágrafo de até 20 palavras. Categorize as respostas no seguintes conteúdos: física, matemática, língua portuguesa ou outros.
Retorne a resposta em um formato json, com as keys: 
fonte: valor deve ser sempre AsimoBot
resposta: a resposta para a pergunta
categoria: a categoria da pergunta
'''

### Formatação das mensagens para realizarmos o Fine Tuning

```
{"messages": 
    [
        {"role": "user", "content": "O que é uma equação quadrática"},
        {"role": "assistant", "content": 
            {
                "resposta": "Equação quadrática é uma equação polinomial de segundo grau, com a forma ax² + bx + c = 0.",
                "categoria": "Matemática",
                "fonte": "GascBot"
            }},
    ]
}

```

### Criando o arquivo JSONL

In [2]:
import json

with open('arquivos/chatbot_respostas.json', encoding="utf8") as f:
    json_respostas = json.load(f)

In [3]:
with open('arquivos/chatbot_respostas.jsonl', 'w', encoding="utf8") as f:
    for entrada in json_respostas:
        resposta = {
            'resposta': entrada['resposta'],
            'categoria': entrada['categoria'],
            'fonte': 'AsimoBot'
        }
        entrada_jsonl = {
            'messages': [
                {'role': 'user', 'content': entrada['pergunta']},
                {'role': 'assistant', 'content': json.dumps(resposta, ensure_ascii=False, indent=2)}
            ]
        }
        json.dump(entrada_jsonl, f, ensure_ascii=False)
        f.write('\n')

In [4]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

In [5]:
file = client.files.create(
    file=open('arquivos/chatbot_respostas.jsonl',  'rb'),
    purpose='fine-tune'
)

client.fine_tuning.jobs.create(
    training_file=file.id,
    model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-Y3ErJ91W8VQRCM4bmlgIlIk5', created_at=1731893836, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Q9xJE0VTannbY3u8McCl2k9j', result_files=[], seed=594595496, status='validating_files', trained_tokens=None, training_file='file-dZp7hcj2EfKcpZOhzEMWXjPz', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [6]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-Y3ErJ91W8VQRCM4bmlgIlIk5', created_at=1731893836, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Q9xJE0VTannbY3u8McCl2k9j', result_files=[], seed=594595496, status='validating_files', trained_tokens=None, training_file='file-dZp7hcj2EfKcpZOhzEMWXjPz', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)], object='list', has_more=False)

### Utilizando o Modelo

In [7]:
mensagens = [{'role': 'user', 'content': 'O que é uma equação quadrática?'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo",
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

Uma equação quadrática é uma equação polinomial de segundo grau, ou seja, uma equação na forma ax^2 + bx + c = 0, onde a, b e c são constantes e a ≠ 0. A incógnita da equação é x e o objetivo é encontrar os valores de x que satisfazem a equação. As equações quadráticas podem ter duas soluções reais, uma solução real ou duas soluções complexas, dependendo do valor do discriminante (b^2 - 4ac).


In [8]:
system_mes = '''
Responda as perguntas em um parágrafo de até 20 palavras. Categorize as respostas no seguintes conteúdos: física, matemática, língua portuguesa ou outros.
Retorne a resposta em um formato json, com as keys: 
fonte: valor deve ser sempre GascBot
resposta: a resposta para a pergunta
categoria: a categoria da pergunta
'''

mensagens = [
    {'role': 'system', 'content': system_mes},
    {'role': 'user', 'content': 'O que é uma equação quadrática?'}
    ]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo",
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

{
    "fonte": "GascBot",
    "resposta": "Uma equação quadrática é uma equação do segundo grau, ou seja, uma equação com a forma ax^2 + bx + c = 0.",
    "categoria": "matemática"
}


In [9]:
resposta.usage

CompletionUsage(completion_tokens=62, prompt_tokens=114, total_tokens=176, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [ ]:
mensagens = [
    {'role': 'user', 'content': 'O que é uma equação quadrática?'}
    ]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="ft:gpt-3.5-turbo-0125:personal::AUlAx92g",
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `ft:gpt-3.5-turbo-0125:personal::971R86c5` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [19]:
resposta.usage

CompletionUsage(completion_tokens=58, prompt_tokens=16, total_tokens=74)